In [ ]:
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
import warnings
warnings.filterwarnings(action='ignore')
!pip install catboost

import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')
import random
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
from sklearn.cluster import KMeans
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

fatal: destination path 'LightGBM' already exists and is not an empty directory.
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Looking for CL_VERSION_2_2
-- Looking for CL_VERSION_2_2 - found
-- Found OpenCL: /usr/lib/x86_64-linux-gnu/libOpenCL.so (found version "2.2") 
-- OpenCL include directory:

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

In [ ]:
### train, test 불러오기
path = '/content/drive/Shareddrives/dacon/404/preprocessed/'
trainset = pd.read_csv(path + 'trainset3.csv')
testset = pd.read_csv(path + 'testset3.csv')

testset.set_index(keys = 'Unnamed: 0', inplace = True)
testset = testset.fillna(-3)

In [ ]:
import re
trainset = trainset.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
testset = testset.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [ ]:
testset.columns = trainset.iloc[:,:-1].columns

In [ ]:
X = trainset.iloc[:,:-1]
y = trainset.iloc[:,-1]

In [ ]:
train_X, valid_X, train_y,  valid_y = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
from sklearn.model_selection import cross_val_predict

In [ ]:
lgbm_param = {}
lgbm_param['max_depth'] = 15
lgbm_param['colsample_bytree'] = 0.9916006382937226
lgbm_param['min_child_samples'] = 141
lgbm_param['min_child_weight'] = 0.1
lgbm_param['num_leaves'] = 11
lgbm_param['reg_alpha'] = 1
lgbm_param['reg_lambda'] = 10
lgbm_param['subsample'] = 0.50239007209780
lgbm_param['max_bin'] = 32
lgbm_param['n_estimators'] = 30000
lgbm_param['learning_rate'] = 0.005
lgbm_param['device'] = 'gpu'
lgbm_param['random_state'] = 42

cat_param = {}
cat_param['task_type'] = 'GPU'
cat_param['max_bin'] = 36
cat_param['iterations'] = 30000
cat_param['random_seed'] = 42
cat_param['auto_class_weights'] = 'Balanced'
cat_param['learning_rate'] = 0.1
cat_param['max_depth'] = 6
cat_param['l2_leaf_reg'] = 9
cat_param['bootstrap_type'] = 'Poisson'

gbc_param = {}
gbc_param['learning_rate'] = 0.1
gbc_param['n_estimators'] = 100
gbc_param['min_samples_split'] = 2
gbc_param['min_samples_leaf'] = 40
gbc_param['min_samples_split'] = 300
gbc_param['max_depth'] = 4
gbc_param['max_features'] = 392
gbc_param['subsample'] = 0.8170162773

In [ ]:
model_1 = LGBMClassifier(**lgbm_param).fit(X, y, verbose=1000)
y_pred = cross_val_predict(model_1, X, y, cv=10, method='predict_proba')
roc_auc_score(y, y_pred[:, 1])
# model_1.fit(train_X, train_y, eval_metric=['AUC'], eval_set=[(valid_X, valid_y)], early_stopping_rounds=5000, verbose=1000)

KeyboardInterrupt: ignored

In [ ]:
train = pd.DataFrame(model_1.predict_proba(X), columns = ['LGBM_0', 'LGBM_1'], index = trainset.index)
test = pd.DataFrame(model_1.predict_proba(testset), columns = ['LGBM_0', 'LGBM_1'], index = testset.index)

In [ ]:
train.to_csv('lgbm_train.csv')
!cp lgbm_train.csv "drive/My Drive/"

test.to_csv('lgbm_test.csv')
!cp lgbm_test.csv "drive/My Drive/"

In [ ]:
model_2 = CatBoostClassifier(**cat_param)

model_2.fit(train_X, train_y, eval_set=[(valid_X, valid_y)], early_stopping_rounds=4000, verbose = 1000, use_best_model=True)

0:	learn: 0.6603279	test: 0.6592212	best: 0.6592212 (0)	total: 32.4ms	remaining: 16m 10s
1000:	learn: 0.2138500	test: 0.5771943	best: 0.5116870 (106)	total: 29.6s	remaining: 14m 16s
2000:	learn: 0.1123987	test: 0.6559693	best: 0.5116870 (106)	total: 59.2s	remaining: 13m 48s
3000:	learn: 0.0663270	test: 0.7341785	best: 0.5116870 (106)	total: 1m 29s	remaining: 13m 21s
4000:	learn: 0.0438265	test: 0.7909294	best: 0.5116870 (106)	total: 1m 58s	remaining: 12m 49s
bestTest = 0.5116869748
bestIteration = 106
Shrink model to first 107 iterations.


In [ ]:
train = pd.DataFrame(model_2.predict_proba(X), columns = ['CAT_0', 'CAT_1'], index = trainset.index)
test = pd.DataFrame(model_2.predict_proba(testset), columns = ['CAT_0', 'CAT_1'], index = testset.index)

train.to_csv('cat_train.csv')
!cp cat_train.csv "drive/My Drive/"

test.to_csv('cat_test.csv')
!cp cat_test.csv "drive/My Drive/"

In [ ]:
model_3 = GradientBoostingClassifier(**gbc_param)
model_3.fit(X,y)
#.fit(train_X, train_y, eval_set=[(valid_X, valid_y)],  eval_metric=['AUC'], early_stopping_rounds=100, verbose = 50, use_best_model=True)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=4,
                           max_features=392, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=40, min_samples_split=300,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=0.8170162773,
                           tol=0.0001, validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
train = pd.DataFrame(model_3.predict_proba(X), columns = ['GBC_0', 'GBC_1'], index = trainset.index)
test = pd.DataFrame(model_3.predict_proba(testset), columns = ['GBC_0', 'GBC_1'], index = testset.index)

train.to_csv('gbc_train.csv')
!cp gbc_train.csv "drive/My Drive/"

test.to_csv('gbc_test.csv')
!cp gbc_test.csv "drive/My Drive/"

In [ ]:
train_1 = pd.DataFrame(model_1.predict_proba(X), columns = ['LGBM_0', 'LGBM_1'], index = trainset.index)
test_1 = pd.DataFrame(model_1.predict_proba(testset), columns = ['LGBM_0', 'LGBM_1'], index = testset.index)

train_2 = pd.DataFrame(model_2.predict_proba(X), columns = ['CAT_0', 'CAT_1'], index = trainset.index)
test_2 = pd.DataFrame(model_2.predict_proba(testset), columns = ['CAT_0', 'CAT_1'], index = testset.index)

train_3 = pd.DataFrame(model_3.predict_proba(X), columns = ['GBC_0', 'GBC_1'], index = trainset.index)
test_3 = pd.DataFrame(model_3.predict_proba(testset), columns = ['GBC_0', 'GBC_1'], index = testset.index)

In [ ]:
k = pd.concat([trainset, train_1, train_2, train_3], axis = 1)
t = k['TARGET']

k.drop('TARGET', axis = 1, inplace = True)

trainset2 = pd.concat([k, t], axis = 1)

In [ ]:
trainset2

,0_t,1_t,2_t,3_t,4_t,5_t,6_t,7_t,8_t,9_t,10_t,11_t,12_t,13_t,14_t,15_t,16_t,17_t,18_t,19_t,20_t,21_t,22_t,23_t,24_t,25_t,26_t,27_t,28_t,29_t,30_t,31_t,32_t,33_t,34_t,35_t,36_t,37_t,38_t,39_t,...,20201127000000,20201127060000,20201127120000,20201127180000,20201128000000,20201128060000,20201128120000,20201128180000,20201129000000,20201129060000,20201129120000,20201129180000,20201130000000,20201130060000,20201130120000,20201130180000,20201201000000,20201201060000,20201201120000,20201201180000,20201202000000,20201202060000,20201202120000,20201202180000,Group_0,Group_1,Group_2,Group_3,Group_4,Group_5,Group_6,model_multi,model_one,LGBM_0,LGBM_1,CAT_0,CAT_1,GBC_0,GBC_1,TARGET
0,-0.461538,-1.0,-1.0,-0.461538,-1.000000,-1.0000,-0.388889,1.187500,0.265255,-0.461538,-4.0,-1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.183844,0.194546,-0.384615,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,12.0,0.0,-0.038365,0.0,-0.384615,...,0.315465,-0.315465,-0.630930,0.101557,0.369070,-0.315465,-0.630930,0.417022,0.315465,0.254692,-0.630930,-0.630930,0.101557,-0.315465,-0.630930,0.101557,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,1,0,0,0,0,0,0,0,1,0.871132,0.128868,0.710593,0.289407,0.851778,0.148222,0
1,-3.000000,-3.0,-3.0,-3.000000,-3.000000,-3.0000,-3.000000,-3.000000,-3.000000,-3.000000,-3.0,-3.000000,-3.0,-3.000000,-3.000000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.00,-3.000000,-3.000000,-3.000000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.000000,-3.0,-3.000000,...,-0.630930,0.796095,2.433031,2.275097,0.254692,0.417022,0.315465,0.658521,0.709142,0.709142,0.684535,0.417022,0.500000,0.101557,0.184535,0.369070,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0,0,1,0,0,0,0,0,1,0.097220,0.902780,0.054967,0.945033,0.127434,0.872566,1
2,-0.307692,0.0,0.0,-0.230769,0.166667,0.3849,0.277778,0.205357,0.052898,-0.230769,8.0,4.618802,2.0,4.666667,8.150555,0.0,0.0,0.0,12.0,6.0,3.0,1.25,0.039395,0.176151,-0.230769,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,-1.0,24.0,1.2,0.594091,1.2,-0.230769,...,0.315465,0.184535,-0.630930,0.101557,-0.630930,-0.630930,-0.630930,-0.630930,-0.130930,0.101557,-0.630930,-0.130930,-0.315465,0.101557,-0.315465,0.000000,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0,1,0,0,0,0,0,0,1,0.781164,0.218836,0.638828,0.361172,0.826242,0.173758,0
3,-3.000000,-3.0,-3.0,-3.000000,-3.000000,-3.0000,-3.000000,-3.000000,-3.000000,-3.000000,-3.0,-3.000000,-3.0,-3.000000,-3.000000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.00,-3.000000,-3.000000,-3.000000,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.000000,-3.0,-3.000000,...,0.369070,0.417022,-0.630930,0.101557,0.369070,0.000000,-0.630930,-0.630930,0.536429,-0.630930,-0.130930,-0.630930,-0.130930,-0.315465,-0.630930,0.000000,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0,1,0,0,0,0,0,0,1,0.926358,0.073642,0.832738,0.167262,0.897152,0.102848,0
4,-0.307692,0.0,0.0,-0.307692,0.000000,0.0000,-0.222222,-0.212500,-0.106568,-0.230769,8.0,4.618802,1.0,12.666667,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.183844,0.030973,-0.230769,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-1.0,0.0,-1.0,0.0,0.0,0.329180,0.0,-0.230769,...,0.500000,0.184535,0.101557,0.101557,-0.130930,0.254692,0.315465,0.000000,0.460399,0.369070,-0.130930,0.000000,-0.130930,0.254692,0.101557,0.460399,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0,0,0,0,1,0,0,0,1,0.479962,0.520038,0.457581,0.542419,0.511012,0.488988,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.307692,0.0,0.0,0.307692,0.000000,0.0000,0.166667,-0.416667,-0.097236,0.307692,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.137883,-0.050813,0

In [ ]:
testset2 = pd.concat([testset, test_1, test_2, test_3], axis = 1)
testset2

,0_t,1_t,2_t,3_t,4_t,5_t,6_t,7_t,8_t,9_t,10_t,11_t,12_t,13_t,14_t,15_t,16_t,17_t,18_t,19_t,20_t,21_t,22_t,23_t,24_t,25_t,26_t,27_t,28_t,29_t,30_t,31_t,32_t,33_t,34_t,35_t,36_t,37_t,38_t,39_t,...,20201126180000,20201127000000,20201127060000,20201127120000,20201127180000,20201128000000,20201128060000,20201128120000,20201128180000,20201129000000,20201129060000,20201129120000,20201129180000,20201130000000,20201130060000,20201130120000,20201130180000,20201201000000,20201201060000,20201201120000,20201201180000,20201202000000,20201202060000,20201202120000,20201202180000,Group_0,Group_1,Group_2,Group_3,Group_4,Group_5,Group_6,model_multi,model_one,LGBM_0,LGBM_1,CAT_0,CAT_1,GBC_0,GBC_1
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
30000,-0.461538,-1.0,-1.0,-0.461538,-1.0,-1.0,-0.444444,0.187500,-1.192347,-0.384615,6.00,-1.000000,1.0,1.666667,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.183844,-0.132599,-0.384615,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-0.2,-0.170820,-0.2,-0.384615,...,1.655537,-0.315465,0.754692,-0.630930,0.101557,-0.130930,0.000000,0.000000,0.184535,0.536429,-0.630930,0.254692,0.000000,-0.630930,-0.630930,-0.630930,0.184535,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0,0,0,0,1,0,0,1,0,0.109736,0.890264,0.016204,0.983796,0.051095,0.948905
30001,0.230769,0.0,0.0,0.230769,0.0,0.0,0.388889,26.703125,73.453272,0.230769,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,65.111421,162.663537,0.307692,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,-1.0,0.0,0.2,0.353875,0.2,0.307692,...,-0.630930,-0.130930,-0.630930,-0.630930,-0.630930,-0.630930,-0.630930,-0.630930,-0.630930,0.184535,-0.630930,-0.630930,-0.630930,-0.630930,-0.630930,-0.630930,0.101557,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,1,0,0,0,0,0,0,0,1,0.783375,0.216625,0.609788,0.390212,0.623300,0.376700
30002,0.692308,0.0,0.0,0.692308,0.0,0.0,1.055556,0.705357,1.909760,0.692308,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.00,0.836490,2.221289,0.769231,0.0,0.0,0.0,0.0,0.0,12.0,5.0,2.0,1.0,-1.0,0.0,1.6,1.737499,1.6,0.769231,...,1.149509,0.536429,0.254692,-0.315465,0.460399,0.754692,0.417022,0.500000,-0.130930,0.500000,0.500000,0.254692,-0.130930,0.536429,0.536429,0.500000,0.254692,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0,0,0,0,0,1,0,0,1,0.488154,0.511846,0.375431,0.624569,0.464438,0.535562
30003,-0.461538,-1.0,-1.0,-0.461538,-1.0,-1.0,-0.111111,0.116071,-0.163823,-0.076923,16.80,3.286335,3.0,12.000000,6.259006,0.0,0.0,0.0,0.0,0.0,0.0,0.50,-0.091922,0.210679,-0.384615,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,3.0,1.0,0.0,0.6,0.957062,0.6,-0.384615,...,-0.315465,-0.630930,-0.130930,-0.630930,-0.315465,0.254692,-0.315465,0.000000,0.254692,-0.630930,0.369070,-0.130930,-0.630930,-0.630930,0.460399,0.101557,0.417022,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0,1,0,0,0,0,0,0,1,0.162863,0.837137,0.054893,0.945107,0.167469,0.832531
30004,0.000000,0.0,0.0,0.000000,0.0,0.0,-0.111111,-0.455357,-0.158522,0.153846,3.75,2.815772,2.0,2.916667,2.872764,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.183844,-0.132599,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,-0.038365,0.0,0.076923,...,0.417022,0.570157,0.417022,0.184535,0.834044,0.000000,0.000000,0.315465,0.973986,0.684535,-0.630930,0.101557,0.834044,0.369070,0.369070,0.101557,0.101557,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,-0.63093,0,0,0,0,0,1,0,1,0,0.098861,0.901139,0.031869,0.968131,0.123051,0.876949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44994,0.384615,0.0,0.0,0.384615,0.0,0.0,0.166667,-0.500000,-0.225512,0.384615,0.00,0.000000,0.0,0.000000,0.00

In [ ]:
trainset2.to_csv('stack_train.csv')
!cp stack_train.csv "drive/My Drive/"

testset2.to_csv('stack_test.csv')
!cp stack_test.csv "drive/My Drive/"